In [9]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from PIL import Image
import os
import numpy as np

from torchvision.transforms import RandomCrop, RandomHorizontalFlip, CenterCrop, RandomVerticalFlip


model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-2]))
model = torch.nn.Sequential(model, torch.nn.AdaptiveAvgPool2d((1, 1)))

model.eval()

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256

In [2]:
from tqdm import tqdm

Image.MAX_IMAGE_PIXELS = 933120000


In [3]:
train_imgs_dirpath = "/projectnb/cs640grp/materials/UBC-OCEAN_CS640/train_images_compressed_80/"
test_imgs_dirpath = "/projectnb/cs640grp/materials/UBC-OCEAN_CS640/test_images_compressed_80/"

train_image_paths = os.listdir(train_imgs_dirpath)
test_image_paths = os.listdir(test_imgs_dirpath)


In [4]:
train_image_paths = [os.path.join(train_imgs_dirpath, pth) for pth in train_image_paths]
test_image_paths = [os.path.join(test_imgs_dirpath, pth) for pth in test_image_paths]

In [5]:
train_feature_dir = 'train_feature_maps_aug'

In [17]:
augmentations = transforms.Compose([
    transforms.RandomChoice([
        transforms.Compose([
            RandomCrop(size=(1250, 1250)),  # Half crop
            transforms.Resize((2500, 2500)),  # Resizing back to original dimensions
        ]),
        RandomHorizontalFlip(),
        RandomVerticalFlip(),
        CenterCrop(size=(1250, 1250)),  # Center crop and resize
        transforms.Resize((2500, 2500)),
    ]),
    transforms.Resize((2500, 2500)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [18]:
# preprocess = transforms.Compose([
#     transforms.Resize((2500, 2500)),  # Resize the image
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

# ])

In [19]:
class AugmentedCustomDataset(Dataset):
    def __init__(self, image_paths, transform=None, augmentations=None, augmentation_factor=10):
        self.image_paths = image_paths
        self.transform = transform
        self.augmentations = augmentations
        self.augmentation_factor = augmentation_factor

    def __len__(self):
        return len(self.image_paths) * self.augmentation_factor

    def __getitem__(self, idx):
        image_path = self.image_paths[idx % len(self.image_paths)]
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        if self.augmentations:
            image = self.augmentations(image)

        return image


In [21]:

# Create dataset and DataLoader
train_dataset = AugmentedCustomDataset(train_image_paths, augmentations=augmentations)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Process in batches
j = 0
for batch in tqdm(train_loader):
    input_batch = batch.to(device)

    with torch.no_grad():
        features = model(input_batch)

    features = features.view(features.size(0), -1).cpu()

    # Save features
    for i, feature in enumerate(features):
        feature_path = os.path.join(train_feature_dir, f"{train_image_paths[j % len(train_image_paths)].split('/')[-1][:-4]}_aug_{j}.pt")
        torch.save(feature, feature_path)
        j += 1

    # Clear memory
    del input_batch, features
    torch.cuda.empty_cache()

print("Feature extraction and saving completed.")

100%|██████████| 1075/1075 [1:55:49<00:00,  6.46s/it] 

Feature extraction and saving completed.
